In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import json, operator, re, pickle , nltk, string, random, statistics, spacy
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
from tqdm import tqdm
from transformers import pipeline

In [ ]:
with open('/content/drive/MyDrive/Thesis/Experiment/Data/Main/user_profiles.pkl', 'rb') as f:
  profiles = pickle.load(f)
# print(len(dataset))

In [ ]:
with open('/content/drive/MyDrive/Thesis/Experiment/Data/Main/user_messages.pkl', 'rb') as f:
  messages = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Thesis/Experiment/Data/Main/user_messages.pkl'

In [ ]:
with open('/content/drive/MyDrive/Thesis/Experiment/Data/Main/entities.pkl', 'rb') as f:
  entity = pickle.load(f)

# **Main Data**

In [ ]:
def read_jsonl_file(file_path):
  """Reads a JSONL file and returns a list of dictionaries."""
  data = []
  with open(file_path, 'r') as f:
    for line in f:
      try:
        data.append(json.loads(line))
      except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
  return data

# =============================
# 1️⃣ Load Dataset
# =============================
file_path = '/content/drive/MyDrive/Perspective Paper/Original_Data/wikipedia.jsonl'
data = read_jsonl_file(file_path)

In [ ]:
wikipedia = defaultdict(list)

for utterance in data:
    wikipedia[utterance['root']].append(utterance)

In [ ]:
wikipedia['262669']

[{'id': '262669',
  'root': '262669',
  'text': 'There\'s some discussion going on regarding Cite News and the other cite templates on [[Template talk:Cite news|that talk page]].  Would love your input :) -- <span style=\\"background-color: #EECCFF;\\">',
  'user': 'SatyrTN',
  'meta': {'is-admin': False},
  'reply-to': None,
  'timestamp': '1.175504160E09'},
 {'id': '262670',
  'root': '262669',
  'text': 'Please see my reply to the ongoing discussion: the very \'\'\'last\'\'\' thing we want is people charging off in all directions using their own \\"pet\\" styles for citations. We want to get the various citation templates in such a condition that \'\'\'anybody\'\'\' can use them to produce useful citations, rather than placing further artificial restrictions and limiting citation to those \\"experts\\" who \\"know how to format citations correctly\\". We need to stop trying to follow any particular existing \\"Manual of Style\\" and synthesise our own: why else did the various \\"Ma

In [ ]:
# Dictionary of all the messages each user sent
user_messages = defaultdict(list)
for root, conversation in wikipedia.items():
  for utterance in conversation:
    user = utterance['user']
    message = utterance['text']
    user_messages[user].append(message)

In [ ]:
with open('/content/drive/MyDrive/Perspective Paper/Original_Data/user_messages.pkl', 'wb') as handle:
    pickle.dump(user_messages, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def generate_histories(conversation):
    history_dict = {}
    for message in conversation:
        current_id = message['id']
        reply_to = message['reply-to']
        if reply_to is None:
            history_dict[current_id] = [current_id]
        else:
            if reply_to in history_dict:
                history_dict[current_id] = history_dict[reply_to] + [current_id]
            else:
                history_dict[current_id] = [reply_to, current_id]
    return history_dict

def extract_dialogue_histories(conversations):
    dialogue_histories = []
    seen_histories = set()  # Use a set to track seen histories

    for conversation in conversations:
        history_dict = generate_histories(conversation)
        for message in conversation:
            current_id = message['id']
            reply_to = message['reply-to']
            if reply_to is not None:  # Skip root messages
                history = tuple(history_dict[current_id][:-1])  # Convert to tuple
                if history not in seen_histories:
                    dialogue_histories.append((history, current_id))
                    seen_histories.add(history)

    return dialogue_histories

# Extract the dialogue histories
dialogue_histories = extract_dialogue_histories(wikipedia.values())


In [ ]:
len(dialogue_histories)

214919

In [ ]:
with open('/content/drive/MyDrive/Perspective Paper/Original_Data/dialogue_histories.pkl', 'rb') as f:
  dialogue_histories = pickle.load(f)

In [ ]:
import random
random.shuffle(dialogue_histories)

total_samples = len(dialogue_histories)
train_split = int(0.8 * total_samples)
test_split = int(0.9 * total_samples)

train_data = dialogue_histories[:train_split]
test_data = dialogue_histories[train_split:test_split]
dev_data = dialogue_histories[test_split:]

print("Train data size:", len(train_data))
print("Test data size:", len(test_data))
print("Dev data size:", len(dev_data))


Train data size: 171935
Test data size: 21492
Dev data size: 21492


In [ ]:
with open('/content/drive/MyDrive/Perspective Paper/Original_Data/train_data.pkl', 'wb') as f:
  pickle.dump(train_data, f)

with open('/content/drive/MyDrive/Perspective Paper/Original_Data/test_data.pkl', 'wb') as f:
  pickle.dump(test_data, f)

with open('/content/drive/MyDrive/Perspective Paper/Original_Data/dev_data.pkl', 'wb') as f:
  pickle.dump(dev_data, f)


In [ ]:
with open('/content/drive/MyDrive/Perspective Paper/Original_Data/train_data.pkl', 'rb') as f:
  train_data = pickle.load(f)

with open('/content/drive/MyDrive/Perspective Paper/Original_Data/test_data.pkl', 'rb') as f:
  test_data = pickle.load(f)

with open('/content/drive/MyDrive/Perspective Paper/Original_Data/dev_data.pkl', 'rb') as f:
  dev_data = pickle.load(f)

print("Train data size:", len(train_data))
print("Test data size:", len(test_data))
print("Dev data size:", len(dev_data))


Train data size: 171935
Test data size: 21492
Dev data size: 21492


# **Preparing Data For Without User Profile Experiment**

In [ ]:
train_data

[(('442505', '442506', '442507'), '442508'),
 (('515773',), '515774'),
 (('126248', '126249', '126250', '126251', '126252', '126253'), '126254'),
 (('63647',), '63648'),
 (('353053', '353054', '353056'), '353057'),
 (('611882', '611883'), '611884'),
 (('144031',), '144032'),
 (('525602', '525603', '525604', '525605', '525606'), '525607'),
 (('512683', '512684', '512685'), '512686'),
 (('358774', '358775', '358776', '358777'), '358778'),
 (('175998',), '175999'),
 (('286374',), '286375'),
 (('61697',), '61698'),
 (('360226', '360227', '360228'), '360229'),
 (('453364', '453366', '453367', '453368'), '453369'),
 (('29435', '29436'), '29437'),
 (('614484', '614485'), '614486'),
 (('104496',), '104497'),
 (('365352', '365353', '365354', '365355'), '365356'),
 (('157719', '157720', '157721'), '157722'),
 (('307873', '307874'), '307875'),
 (('319406',), '319407'),
 (('81201',), '81202'),
 (('205880',), '205881'),
 (('537396', '537397'), '537398'),
 (('589666', '589667'), '589668'),
 (('50803

In [ ]:
# ======================================================================================================
# This function returns last user in the conversation (the one we are going to select the response for)
# ======================================================================================================

def get_user(id_histories, dataset):
  response_user_map = {}
  for user, conversation in dataset.items():
      for utterance in conversation:
          response_user_map[utterance['id']] = utterance['user']
  response_user = []
  for history, response_id in id_histories:
      if response_id in response_user_map:
          response_user.append(response_user_map[response_id])

  return response_user

In [ ]:
train_response_user = get_user(train_data, wikipedia)
dev_response_user = get_user(dev_data, wikipedia)
test_response_user = get_user(test_data, wikipedia)

In [ ]:
# =============================================================
# Creating Distractor list among same user's all responses.
# =============================================================
def get_distractor(response_user):
  distractor_list = []
  for user in response_user:
    distractor = random.choice(list(user_messages[user]))
    distractor_list.append(distractor)

  return distractor_list

In [ ]:
# ========================================================
# This function maps the ids in the DH to their real texts
# ========================================================
def id_to_text(id_histories, dataset):
    utterance_dict = {}
    for _ , conversation in dataset.items():
        for utterance in conversation:
            utterance_dict[utterance['id']] = utterance['text']

    text_histories = []
    for history, response_id in id_histories:
        text_history = [utterance_dict[utt_id] for utt_id in history if utt_id in utterance_dict]
        true_response = utterance_dict.get(response_id, "")
        text_history = '__eou__'.join(text_history)
        text_histories.append((text_history, true_response))


    return text_histories, utterance_dict

In [ ]:
def generate_final_data(text_histories, wrong_response):
  final_list = []
  for (history, correct_response), wrong_response in zip(text_histories, wrong_response):
      # Randomly choose between the correct and wrong response
      if random.choice([True, False]):
          chosen_response = correct_response
          label = 1  # 1 if true response chosen
      else:
          chosen_response = wrong_response
          label = 0  # 0 if wrong response chosen
      # Append a tuple to the final list: (history, chosen_response, label)
      final_list.append((history, chosen_response, label))

  return final_list

In [ ]:
train_distractor = get_distractor(train_response_user)
dev_distractor = get_distractor(dev_response_user)
test_distractor = get_distractor(test_response_user)

In [ ]:
train_text_histories, utterance_dict = id_to_text(train_data, wikipedia)
dev_text_histories, utterance_dict = id_to_text(dev_data, wikipedia)
test_text_histories, utterance_dict = id_to_text(test_data, wikipedia)

In [ ]:
final_train_data = generate_final_data(train_text_histories, train_distractor)
final_dev_data = generate_final_data(dev_text_histories, dev_distractor)

In [ ]:
final_test_data = [
    [history[0], history[1], wrong_response]
    for history, wrong_response in zip(test_text_histories, test_distractor)
]

In [ ]:
# These are the data for response selection with distractor choosed among user's prev messages.

with open('/content/drive/MyDrive/Perspective Paper/Original_Data/final_train.pkl', 'wb') as f:
  pickle.dump(final_train_data, f)

with open('/content/drive/MyDrive/Perspective Paper/Original_Data/final_dev.pkl', 'wb') as f:
  pickle.dump(final_dev_data, f)

with open('/content/drive/MyDrive/Perspective Paper/Original_Data/final_test.pkl', 'wb') as f:
  pickle.dump(final_test_data, f)

## **Preparing Data For With UserProfile Experiment**

In [ ]:
with open('/content/drive/MyDrive/Thesis/Experiment/Data/Experiment1/TrainSet.pkl', 'rb') as f:
  Ex1_train = pickle.load(f)

with open('/content/drive/MyDrive/Thesis/Experiment/Data/Experiment1/DevSet.pkl', 'rb') as f:
  Ex1_dev = pickle.load(f)

with open('/content/drive/MyDrive/Thesis/Experiment/Data/Experiment1/TestSet.pkl', 'rb') as f:
  Ex1_test = pickle.load(f)

In [ ]:
def generate_datase(data, response_user):
  merged_dataset = []
  for main, user in zip(data, response_user):
    user_profile = user_profiles[user]
    profile_text = ', '.join([f"{key}:{value}" for key, value in user_profile.items()])
    common_words = ' '.join(user_entity[user])
    merged_dataset.append((f"{main[0]}|Speaker Profile: {profile_text}|Speaker Frequent Words: {common_words}", main[1], main[2]))

  return merged_dataset

In [ ]:
trainset = generate_datase(Ex1_train, train_response_user)
devset = generate_datase(Ex1_dev, dev_response_user)
testset = generate_datase(Ex1_test, test_response_user)

In [ ]:
with open('/content/drive/MyDrive/Thesis/Experiment/Data/Experiment1/TrainSet_SUP.pkl', 'wb') as f:
  pickle.dump(trainset, f)

with open('/content/drive/MyDrive/Thesis/Experiment/Data/Experiment1/DevSet_SUP.pkl', 'wb') as f:
  pickle.dump(devset, f)

with open('/content/drive/MyDrive/Thesis/Experiment/Data/Experiment1/TestSet_SUP.pkl', 'wb') as f:
  pickle.dump(testset, f)